In [1]:
import pandas as pd
import sys
sys.path.append('..')
from processing_helper_functions import *

articles = pd.read_csv('validated_qiushi_articles.csv')
print(f"Validated articles: {len(articles)}")
articles.head(1)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/calebharding/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Validated articles: 1998


,authors,title,date,year,edition,url,text,quarter,year_quarter
0,习近平,辩证唯物主义是中国共产党人的世界观和方法论,2018-12-31,2019,1.0,http://www.qstheory.cn/dukan/qs/2018-12/31/c_1...,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日，纪念马克思诞辰200周年...,1,2019 Q1


In [2]:
# remove punctuation
articles['cleaned_text'] = articles['text'].map(remove_punctuation)
articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text
0,习近平,辩证唯物主义是中国共产党人的世界观和方法论,2018-12-31,2019,1.0,http://www.qstheory.cn/dukan/qs/2018-12/31/c_1...,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日，纪念马克思诞辰200周年...,1,2019 Q1,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日纪念马克思诞辰200周年大...


In [ ]:
# add spaces to the text, necessary for some analysis
articles['cleaned_text_wspaces'] = articles["cleaned_text"].map(lambda x: parse_chinese(x))
articles.head(1)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/r4/qqn8tv4d7g18t_5fsxt6y1340000gn/T/jieba.cache
Loading model cost 0.295 seconds.
Prefix dict has been built successfully.


,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text,cleaned_text_wspaces
0,习近平,辩证唯物主义是中国共产党人的世界观和方法论,2018-12-31,2019,1.0,http://www.qstheory.cn/dukan/qs/2018-12/31/c_1...,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日，纪念马克思诞辰200周年...,1,2019 Q1,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日纪念马克思诞辰200周年大...,辩证唯物主义 是 中国共产党 人 的 世界观 和 方法论 习近平 2018 年 5 月 4 ...


In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/calebharding/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
def get_ngrams(text, n):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ''.join(grams) for grams in n_grams]

articles['unigram'] = articles['cleaned_text_wspaces'].map(lambda x: get_ngrams(x, 1))

In [6]:
articles['bigram'] = articles['cleaned_text_wspaces'].map(lambda x: get_ngrams(x, 2))

In [7]:
articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text,cleaned_text_wspaces,unigram,bigram
0,['习近平'],辩证唯物主义是中国共产党人的世界观和方法论,2018-12-31,2019,1.0,http://www.qstheory.cn/dukan/qs/2018-12/31/c_1...,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日，纪念马克思诞辰200周年...,1,2019 Q1,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日纪念马克思诞辰200周年大...,辩证唯物主义 是 中国共产党 人 的 世界观 和 方法论 习近平 2018 年 5 月 4 ...,"[辩证唯物主义, 是, 中国共产党, 人, 的, 世界观, 和, 方法论, 习近平, 201...","[辩证唯物主义是, 是中国共产党, 中国共产党人, 人的, 的世界观, 世界观和, 和方法论..."


In [8]:
# create unigram count dictionary
def ngram_counts(ngram):
    # create word count dictionary
    word_count = {}

    for word in ngram:
        if word in word_count.keys():
            cur_val = word_count[word]
            cur_val += 1
            word_count[word] = cur_val
        else:
            word_count[word] = 1

    # sort the dictionary
    word_count = {k: v for k, v in sorted(word_count.items(), key=lambda item: item[1], reverse=True)}

    return word_count

articles['unigram_count'] = articles["unigram"].map(lambda x: ngram_counts(x))
articles['bigram_count'] = articles["bigram"].map(lambda x: ngram_counts(x))

articles.head(1)

,authors,title,date,year,edition,url,text,quarter,year_quarter,cleaned_text,cleaned_text_wspaces,unigram,bigram,unigram_count,bigram_count
0,['习近平'],辩证唯物主义是中国共产党人的世界观和方法论,2018-12-31,2019,1.0,http://www.qstheory.cn/dukan/qs/2018-12/31/c_1...,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日，纪念马克思诞辰200周年...,1,2019 Q1,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日纪念马克思诞辰200周年大...,辩证唯物主义 是 中国共产党 人 的 世界观 和 方法论 习近平 2018 年 5 月 4 ...,"[辩证唯物主义, 是, 中国共产党, 人, 的, 世界观, 和, 方法论, 习近平, 201...","[辩证唯物主义是, 是中国共产党, 中国共产党人, 人的, 的世界观, 世界观和, 和方法论...","{'的': 139, '和': 54, '我们': 32, '矛盾': 32, '是': 2...","{'客观实际': 8, '的矛盾': 7, '矛盾的': 7, '世界观和': 6, '和方..."


In [9]:
articles.to_csv('processed_qiushi_articles.csv', index=False)